In [1]:
"""
    @Author: Vipul Shete
    @Date: 2022-12-27
    @Last Modified by: Vipul Shete
    @Last Modified Date: 2022-12-28
    @Title : Stock Market Details, company overview and flow cash (UC-3).
"""

'\n    @Author: Vipul Shete\n    @Date: 2022-12-27\n    @Last Modified by: Vipul Shete\n    @Last Modified Date: 2022-12-28\n    @Title : Stock Market Details, company overview and flow cash (UC-3).\n'

In [2]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
from pyspark.sql import SparkSession
import requests

In [3]:
spark = SparkSession.builder\
                    .master("local")\
                    .appName('stock_market_UC3')\
                    .getOrCreate()

22/12/28 11:02:25 WARN Utils: Your hostname, vipul-HP-Pavilion-Laptop-15-cs2xxx resolves to a loopback address: 127.0.1.1; using 192.168.43.246 instead (on interface wlx001ea63c098f)
22/12/28 11:02:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/28 11:02:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [12]:
def daily_stock_price(company_name, date):
    """
        Description:
            Get the stock daily status using alpha vantage (open , high, low, close values)
        Parameter:
            company_name: name of company (symbol used in alpha vantage docomenation)
            date: date of records.
        Return: 
            stock details open, high, low, close 
    """
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={company_name}&apikey=3ZN85270ZOKUBGMP"
    r = requests.get(url)
    data = r.json()
    date_stock_price = data["Time Series (Daily)"][date]
    
    open = date_stock_price["1. open"]
    high = date_stock_price["2. high"]
    low = date_stock_price["3. low"]
    close = date_stock_price["4. close"]
    
    return open, high, low, close

def company_overview(company_name):
    """
        Description:
            Get the company overview using alpha vantage (pe_ratio and gross_profit) year data
        Parameter:
            company_name: name of company (symbol used in alpha vantage docomenation)
            date: date of records.
        Return: 
            stock overview pe_ratio, gross_profit
    """
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={company_name}&apikey=3ZN85270ZOKUBGMP"
    r = requests.get(url)
    data = r.json()
    
    pe_ratio = data["PERatio"]
    gross_profit = data["GrossProfitTTM"]

    return pe_ratio, gross_profit

def cash_flow(company_name):
    """
        Description:
            Get the company cash flow using alpha vantage (operating_cashflow, profit_loss, net_income) year data
        Parameter:
            company_name: name of company (symbol used in alpha vantage docomenation)
        Return: 
            stock cash flow operating_cashflow, profit_loss, net_income
    """
    url = f"https://www.alphavantage.co/query?function=CASH_FLOW&symbol={company_name}&apikey=3ZN85270ZOKUBGMP"
    r = requests.get(url)
    data = r.json()
    
    anaunal_report = data["annualReports"][0]
    
    operating_cashflow = anaunal_report["operatingCashflow"]
    profit_loss = anaunal_report["profitLoss"]
    net_income = anaunal_report["netIncome"]


    return operating_cashflow, profit_loss, net_income


def company_details(company_name, date):
    """
        Description:
            make a dataframe of single company
        Parameter:
            company_name: name of company (symbol used in alpha vantage docomenation)
            date: date of records.
        Return: 
            dataframe 
    """
    open, high, low, close = daily_stock_price(company_name, date)
    pe_ratio, gross_profit = company_overview(company_name)
    operating_cashflow, profit_loss, net_income = cash_flow(company_name)

    columns = ["Company_name", "Open", "High", "Low", "Close", "PE_Ratio", "Gross_profit", "Operating_Cashflow", "Profit_Loss", "Net_Income"]
    values = [(company_name, open, high, low, close, pe_ratio, gross_profit, operating_cashflow, profit_loss, net_income)]

    stock_info = spark.createDataFrame(values, columns)
    
    return stock_info

In [6]:
company_name = "APLE"
date = "2022-12-23"
apple = company_details(company_name, date)
apple.show()

+------------+-----+-----+------+-----+--------+------------+------------------+-----------+----------+
|Company_name| Open| High|   Low|Close|PE_Ratio|Gross_profit|Operating_Cashflow|Profit_Loss|Net_Income|
+------------+-----+-----+------+-----+--------+------------+------------------+-----------+----------+
|        APLE|15.57|15.86|15.465|15.83|   23.38|   399545000|         217562000|   18828000|  18828000|
+------------+-----+-----+------+-----+--------+------------+------------------+-----------+----------+



In [7]:
company_name = "DPZ"
date = "2022-12-23"
dominos_pizza = company_details(company_name, date)
dominos_pizza.show()

+------------+------+------+-----+------+--------+------------+------------------+-----------+----------+
|Company_name|  Open|  High|  Low| Close|PE_Ratio|Gross_profit|Operating_Cashflow|Profit_Loss|Net_Income|
+------------+------+------+-----+------+--------+------------+------------------+-----------+----------+
|         DPZ|351.35|354.34|347.0|353.02|   28.58|  1208741000|         592794000|  491296000| 491296000|
+------------+------+------+-----+------+--------+------------+------------------+-----------+----------+



In [21]:
company_name = "MCD"
date = "2022-12-23"
mc_donalds = company_details(company_name, date)
mc_donalds.show()

+------------+-----+-------+------+------+--------+------------+------------------+-----------+----------+
|Company_name| Open|   High|   Low| Close|PE_Ratio|Gross_profit|Operating_Cashflow|Profit_Loss|Net_Income|
+------------+-----+-------+------+------+--------+------------+------------------+-----------+----------+
|         MCD|265.9|268.345|264.79|267.57|   33.74| 12580200000|        9141500000| 8907700000|7545200000|
+------------+-----+-------+------+------+--------+------------+------------------+-----------+----------+



In [18]:
company_name = "ACN"
date = "2022-12-23"
accenture = company_details(company_name, date)
accenture.show()

+------------+------+------+--------+------+--------+------------+------------------+-----------+----------+
|Company_name|  Open|  High|     Low| Close|PE_Ratio|Gross_profit|Operating_Cashflow|Profit_Loss|Net_Income|
+------------+------+------+--------+------+--------+------------+------------------+-----------+----------+
|         ACN|265.79|266.71|263.3079|266.09|   24.17| 19701539000|        9541129000|16185127000|6877169000|
+------------+------+------+--------+------+--------+------------+------------------+-----------+----------+



In [15]:
company_name = "TCS"
date = "2022-12-23"
tata_con_services = company_details(company_name, date)
tata_con_services.show()

+------------+----+----+------+-----+--------+------------+------------------+-----------+----------+
|Company_name|Open|High|   Low|Close|PE_Ratio|Gross_profit|Operating_Cashflow|Profit_Loss|Net_Income|
+------------+----+----+------+-----+--------+------------+------------------+-----------+----------+
|         TCS|4.18|4.32|4.1514| 4.26|   3.341|   636237000|          56990000|   81718000|  81718000|
+------------+----+----+------+-----+--------+------------+------------------+-----------+----------+



## merage data frame

In [22]:
appended_one = apple.union(dominos_pizza)
appended_two = appended_one.union(mc_donalds)
appended_three = appended_two.union(accenture)
company_details = appended_three.union(tata_con_services)

In [23]:
company_details.show()

+------------+------+-------+--------+------+--------+------------+------------------+-----------+----------+
|Company_name|  Open|   High|     Low| Close|PE_Ratio|Gross_profit|Operating_Cashflow|Profit_Loss|Net_Income|
+------------+------+-------+--------+------+--------+------------+------------------+-----------+----------+
|        APLE| 15.57|  15.86|  15.465| 15.83|   23.38|   399545000|         217562000|   18828000|  18828000|
|         DPZ|351.35| 354.34|   347.0|353.02|   28.58|  1208741000|         592794000|  491296000| 491296000|
|         MCD| 265.9|268.345|  264.79|267.57|   33.74| 12580200000|        9141500000| 8907700000|7545200000|
|         ACN|265.79| 266.71|263.3079|266.09|   24.17| 19701539000|        9541129000|16185127000|6877169000|
|         TCS|  4.18|   4.32|  4.1514|  4.26|   3.341|   636237000|          56990000|   81718000|  81718000|
+------------+------+-------+--------+------+--------+------------+------------------+-----------+----------+

